In [ ]:
import os
import requests
import json
import gpxpy
import folium
import geojson
from shapely.geometry import LineString
from folium import plugins
from math import sin, cos, sqrt, atan2, radians

# hard wire search criteria for routes near Lambfair Green distance is in metres
lat, lon, dist = 52.07597668709528, 0.7173702199141314, 200
directory = 'tracks'

# refresh our list of tracks from RWGPS
refresh_rwgps_routes( directory )

# OK - now we loop through our tracks and look for one that's near our destination...
matches = find_close_routes ( directory, lat, lon, dist )	
#matches = ['36216168.gpx', '32408351.gpx', '43141887.gpx', 'Back_to_Brinkley.gpx', '11764387.gpx', '11775438.gpx', '35648012.gpx', '35592301.gpx']

print (f"Matched tracks: {matches}")

# Now make a map with the selected routes on it...
map = make_folium_map ( "tracks", matches, "my_map.html" )
map



## Functions

In [1]:
def refresh_rwgps_routes ( directory = 'tracks', user:int = 657096, api_key = '', auth_token = '' ) -> list:
    # Download routes for a given user from RWGPS
    # Downloads into directory ".\tracks" relative to CWD by default, only if the GPX file isn't already there
    # If you have a developer token, this can check many routes, otherwise the public interface seems to load the most recent 25
    # (see https://ridewithgps.com/api)
    if not(os.path.exists(directory)):
        os.mkdir(directory)

    # get list of existing routes
    files = []
    for file in os.listdir(directory):
        if os.path.isfile(os.path.join(directory, file)) and file.endswith('.gpx') :
            files.append(file)
    print (f"Found {len(files)} existing routes in directory '{directory}'...")
    
    # Now, load the latest routes from RWGPS - public method seems to get latest 25 routes... User 657096 is me!
    if api_key == "":
        r = requests.get (f"https://ridewithgps.com/users/{user}/routes.json" )
    else:
        #...or call with credentials can take parameters, but need auth token...
        r = requests.get (f"https://ridewithgps.com/users/{user}/routes.json", params={"offset" : "0", "limit" : "500", "version": "2", "apikey": api_key, "auth_token": auth_token } )

    if (r.status_code == 200) :
        if (api_key == ""):
            routes = json.loads(r.content)
        else:
            # return structure is different in authenticated call!
            routes = json.loads(r.content)['results']
    else:
        print(f"Error: {r.status_code} - {r.content}")
        
    # Loop through the routes, and download if we don't have it
    print(f"Checking {len(routes)} routes from RWGPS for missing routes")
    for route in routes:
        id = f"{route['id']}.gpx"
        if (id not in files):
            print(f'Route: {id} does not exist - downloading GPX...')
            r = requests.get (f"https://ridewithgps.com/routes/{id}")
            if (r.status_code == 200):
                with open(f"tracks\\{id}", "wb") as file:
                    file.write(r.content)
                files.append( str(id) )
            else:
                print(f'Failed to get route {id}: {r.status_code}')
    
    return files

In [2]:
def calculate_distance(lat1, lon1, lat2, lon2) -> float:
    # Use Haversine formula to caucluate distance between 2 points in meters
    # approximate radius of Earth in meters
    R = 6371000

    # convert decimal degrees to radians
    lat1_rad = radians(lat1)
    lon1_rad = radians(lon1)
    lat2_rad = radians(lat2)
    lon2_rad = radians(lon2)

    # haversine formula
    dlon = lon2_rad - lon1_rad
    dlat = lat2_rad - lat1_rad
    a = sin(dlat / 2) ** 2 + cos(lat1_rad) * cos(lat2_rad) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

In [37]:
def load_gpx_from_file ( filename:str ) -> dict:
    # load GPX file into dictionary
    # returns dictionary - {link:str, name:str, length:float, uphill:float, midpoint:(lat,lon), points: [(lat,lon),(lat,lon)...]}    with open(filename, 'r') as gpx_file:
    with open(filename, "r") as gpx_file:
        gpx = gpxpy.parse(gpx_file)

    gpx_return = dict(name = gpx.name, link = gpx.link, length = gpx.length_3d(), uphill = gpx.get_uphill_downhill(), points = [])
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                gpx_return['points'].append((point.latitude, point.longitude))
    num_points = len(gpx_return['points'])
    gpx_return['midpoint'] = gpx_return['points'][num_points//2]
    return gpx_return

In [103]:
def find_close_routes ( directory:str, lat:float, lon:float, dist:float ) -> set:

	matched_routes = set() 	# a set to avoid duplicates when adding route matches...
	print(f"Now checking for track that's within {dist}m of lat:{lat}, lon:{lon}")
	num_tracks = len(os.listdir(directory))
	count = 0
	for file in os.listdir(directory):
		count += 1
		if os.path.isfile(os.path.join(directory, file)) and file.endswith('.gpx') :
			print(f"Checking files: {file} - {count / num_tracks:.1%}", end="\r")
			with open(os.path.join(directory, file), 'r') as gpx_file:
				gpx = gpxpy.parse(gpx_file)
				for track in gpx.tracks:
					for segment in track.segments:
						for point in segment.points:
							distance = calculate_distance(lat, lon, point.latitude, point.longitude)
							#print (f"lat:{point.latitude}, lon:{point.longitude}, distance:{distance}")
							if distance < dist:
								#print (f"Route {file} ({gpx.name}) matched! - distance:{distance}m")
								matched_routes.add(file)
	return list(matched_routes)

In [99]:
def make_folium_map( directory:str = './', file_list:list = [], map_name='my_map.html', zoom_level=12):
# assumes input is list of file roots
    for file in file_list:
        route = load_gpx_from_file ( os.path.join(directory, file))
		
        print('data created for ' + file)

        #get start and end lat/long
        lat_start, long_start = route['points'][0]
        lat_end, long_end = route['points'][-1]
        not_a_loop = calculate_distance(lat_start, long_start, lat_end, long_end) > 500 # less than 500m means we're probably a loop route
        
        activity_color = ['red', 'blue', 'green', 'orange', 'pink', 'purple', 'gray'][file_list.index(file)%7]
        activity_icon='bicycle'
        
        #first time through, we create the map - after that, we're just adding lines to it...
        if file_list.index(file) == 0:
            mymap = folium.Map( location=[ lat_start, long_start ], zoom_start=zoom_level, tiles=None)
            folium.TileLayer('openstreetmap', name='OpenStreet Map').add_to(mymap)
            folium.TileLayer('https://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}', attr="Tiles &copy; Esri &mdash; National Geographic, Esri, DeLorme, NAVTEQ, UNEP-WCMC, USGS, NASA, ESA, METI, NRCAN, GEBCO, NOAA, iPC", name='Nat Geo Map').add_to(mymap)
            folium.TileLayer('http://tile.stamen.com/terrain/{z}/{x}/{y}.jpg', attr="terrain-bcg", name='Terrain Map').add_to(mymap)
            folium.LayerControl().add_to(mymap)

            #fullscreen option
            plugins.Fullscreen( position='topright' ).add_to(mymap)

        html_hint = f"<a href='{route['link']}'>{route['name']} Length:{route['length']/1000:.1f}</a>"

        #draw line
        folium.PolyLine(route['points'], popup=folium.Popup(html_hint), color=activity_color, weight=4.5, opacity=.5).add_to(mymap)
    
        #build starting marker
        folium.vector_layers.CircleMarker(location=[lat_start, long_start], radius=9, color=activity_color, weight=1, fill_color=activity_color, fill_opacity=1, popup=folium.Popup(html_hint)).add_to(mymap) 
        #Overlay triangle
        folium.RegularPolygonMarker(location=[lat_start, long_start], fill_color='white', fill_opacity=1, color='white', number_of_sides=3, radius=3, rotation=0, popup=folium.Popup(html_hint)).add_to(mymap)
        
        #end marker if we're not a loop
        if not_a_loop :
            folium.vector_layers.CircleMarker(location=[lat_end, long_end], radius=9, color='red', weight=1, fill_color='red', fill_opacity=1,  popup=popup).add_to(mymap)
            #Overlay square
            folium.RegularPolygonMarker(location=[lat_end, long_end], fill_color='white', fill_opacity=1, color='white', number_of_sides=4, radius=3, rotation=45, popup=popup).add_to(mymap)
        
        #add 'mid' marker with link to RWGPSRoute
        #folium.Marker(route['midpoint'],tooltip=html_hint, icon=folium.Icon(color=activity_color, icon_color='white', icon=activity_icon, prefix='fa')).add_to(mymap)
    
 
    mymap.save(map_name) # saves to html file for display below